In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torch.autograd import Variable

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU for training.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU for training.")

# Define the Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(256, 3, kernel_size=3, padding=1)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Hyperparameters
learning_rate = 0.001
batch_size = 32
num_epochs = 1

# Load CIFAR-10 dataset
train_dataset = CIFAR10(root='./data', train=True, transform=ToTensor(), download=True)
test_dataset = CIFAR10(root='./data', train=False, transform=ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create the Autoencoder model
autoencoder = Autoencoder().to(device)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    train_loss = 0.0

    for images, _ in train_loader:
        images = images.to(device)
        reconstructions = autoencoder(images)


        # Compute loss
        loss = criterion(reconstructions, images)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    # Calculate average loss
    train_loss /= len(train_loader.dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}")

import os

save_dir = 'save'
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, 'cnn_autoencoder.pth')

# Save the trained model
torch.save(autoencoder.state_dict(), save_path)

# Load the trained model
autoencoder = Autoencoder().to(device)
autoencoder.load_state_dict(torch.load('save/cnn_autoencoder.pth'))


# Get encoder features
encoder = autoencoder.encoder
encoder_features_train = []
labels_train = []  # Store the labels

for images, labels in train_loader:
    images = images.to(device)  # Move images to the GPU device
    features = encoder(images)
    encoder_features_train.append(features.detach())
    labels_train.extend(labels)  # Save the labels

encoder_features_train = torch.cat(encoder_features_train, dim=0)

# Reshape encoder features
encoder_features_train = encoder_features_train.view(encoder_features_train.size(0), -1)

# Convert labels_train to a NumPy array
labels_train = np.array(labels_train)